# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission, if necessary. Sections that begin with **'Implementation'** in the header indicate where you should begin your implementation for your project. Note that some sections of implementation are optional, and will be marked with **'Optional'** in the header.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [1]:
# Load pickled data
import pickle

# TODO: Fill this in based on where you saved the training and testing data

training_file = "traffic-signs-data/train.p"
testing_file = "traffic-signs-data/test.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

from keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

Using TensorFlow backend.


In [2]:
import numpy as np
print(X_train.shape)
print(y_train.shape)
print(set(y_train.reshape(y_train.shape[0])))
n_classes = len(set(y_train.reshape(y_train.shape[0])))
print(n_classes)

y_train = y_train.reshape(y_train.shape[0])
print(y_train.shape)

(50000, 32, 32, 3)
(50000, 1)
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
10
(50000,)


---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 2D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below.

In [ ]:
### Replace each question mark with the appropriate value.

# TODO: Number of training examples
n_train = X_train.shape[0]

# TODO: Number of testing examples.
n_test = X_test.shape[0]

# TODO: What's the shape of an traffic sign image?
image_shape = X_train.shape[1:]

# TODO: How many unique classes/labels there are in the dataset.
n_classes = len(set(y_train))

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc.

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections.

In [ ]:
### Data exploration visualization goes here.
### Feel free to use as many code cells as needed.
import matplotlib.pyplot as plt
# Visualizations will be shown in the notebook.
%matplotlib inline

# store the label, sign name, and sample image of each traffic sign
sample_data = {}

# plot the the number of samples for each traffic sign
plt.figure(figsize=(10,5))
plt.hist(y_train, bins=n_classes)
plt.xlabel('class labels')
plt.ylabel('number of samples')
plt.title('Class Sample Distribution in Training Set')
plt.show()

# get the sign names
import pandas as pd
df = pd.read_csv('signnames.csv')

# show an example of each class of traffic sign
from textwrap import wrap
from collections import Counter
label_counter = Counter(y_train)
fig = plt.figure(figsize=(10,15))
idx = 0 #image index
for label,samples in label_counter.items():
    image = X_train[idx].squeeze()
    ax = fig.add_subplot(8,8,label+1)
    ax.imshow(image)
    title = df.iloc[label,1]
    #wrap the title
    ax.set_title("\n".join(wrap(title, 12)))   
    ax.set_xticks(())
    ax.set_yticks(())
    idx += samples
    sample_data[label] = (title,image)
plt.tight_layout()

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

There are various aspects to consider when thinking about this problem:

- Neural network architecture
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

**NOTE:** The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play!

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:
### Preprocess the data here.
### Feel free to use as many code cells as needed.

In [ ]:
import numpy as np
from scipy.ndimage.interpolation import rotate

# determine the number of samples per each class
avg_sample_count = n_train / n_classes

# create additional data for under represented classes by rotating the images
additional_samples = np.empty([0,32,32,3])
additional_labels = np.empty([0])
current = 0
for label,no_of_samples in label_counter.items():
    # if number of samples for the class is less than average / 2, create more samples by rotating images
    print("process label", label, "with samples", no_of_samples)
    if no_of_samples < avg_sample_count / 2:
        begin = current
        end = current + no_of_samples
        print("adding", 3*(end-begin), "samples", "for label", label)
        r1 = rotate(X_train[begin:end], axes=(1,2), angle=15, reshape=False)
        r2 = rotate(X_train[begin:end], axes=(1,2), angle=30, reshape=False)
        r3 = rotate(X_train[begin:end], axes=(1,2), angle=-15, reshape=False)
        additional_samples = np.concatenate([additional_samples, r1, r2, r3])
        additional_labels = np.concatenate([additional_labels, y_train[begin:end], y_train[begin:end], y_train[begin:end]])
    current = current + no_of_samples

X_train = np.concatenate([X_train, additional_samples])
y_train = np.concatenate([y_train, additional_labels])

# plot the new class distribution
plt.figure(figsize=(10,5))
plt.hist(y_train, bins=n_classes)
plt.xlabel('class labels')
plt.ylabel('number of samples')
plt.title('New Class Distribution')
plt.show()

In [3]:
# zero center the data
X_train = X_train - np.mean(X_train)
X_test = X_test - np.mean(X_test)

# normalize the data
X_train = X_train / np.std(X_train)
X_test = X_test / np.std(X_test)

### Question 1 

_Describe how you preprocessed the data. Why did you choose that technique?_

**Answer:**

We apply mean substraction by substracting the mean across every feature. We also normalize the data by dividing it by the standard deviation. This common preprocessing technique can rearrange data to have similar scale near the center.

In [4]:
### Generate data additional data (OPTIONAL!)
### and split the data into training/validation/testing sets here.
### Feel free to use as many code cells as needed.

from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

### Question 2

_Describe how you set up the training, validation and testing data for your model. **Optional**: If you generated additional data, how did you generate the data? Why did you generate the data? What are the differences in the new dataset (with generated data) from the original dataset?_

**Answer:**
Use 20% of the training data for validation. Use remaining 80% of the training data for training.

For classes of traffic sign that are under represented in the training data, I created additional data by rotating the images. The additional data will result in more robust learning and protect against simliar deformations in test data.

In [5]:
### Define your architecture here.
### Feel free to use as many code cells as needed.
from sklearn.utils import shuffle
import tensorflow as tf
EPOCHS = 10
BATCH_SIZE = 32
KEEP_PROB = 0.5

from tensorflow.contrib.layers import flatten

def LeNet(x, keep_prob):
    # Hyperparameters
    mu = 0
    sigma = 0.1
    
    # Layer 1: Convolutional. Input = 32x32x3. Output = 28x28x6
    w1 = tf.Variable(tf.truncated_normal(shape=(5,5,3,6), mean = mu, stddev = sigma))
    b1 = tf.Variable(tf.zeros(6))
    conv1 = tf.nn.conv2d(x, w1, strides=[1,1,1,1], padding='VALID')
    conv1 = tf.nn.bias_add(conv1, b1)
    conv1 = tf.nn.relu(conv1)
    #Pooling. Input = 28x28x6. Output = 14x14x6.
    conv1 = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
    
    # Layer 2: Convolutional. Output = 10x10x16.
    w2 = tf.Variable(tf.truncated_normal(shape=(5,5,6,16), mean = mu, stddev = sigma))
    b2 = tf.Variable(tf.zeros(16))
    conv2 = tf.nn.conv2d(conv1, w2, strides=[1,1,1,1], padding='VALID')
    conv2 = tf.nn.bias_add(conv2, b2)    
    conv2 = tf.nn.relu(conv2)
    conv2 = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')

    # Flatten. Input = 5x5x16. Output = 400.
    conv2 = flatten(conv2)
    
    # Layer 3: Fully Connected. Input = 400. Output = 120.
    w3 = tf.Variable(tf.truncated_normal(shape=(400,120), mean = mu, stddev = sigma))
    b3 = tf.Variable(tf.zeros(120))
    fc1 = tf.add(tf.matmul(conv2, w3), b3)
    fc1 = tf.nn.relu(fc1)    
    
    # Dropout 1
    drop1 = tf.nn.dropout(fc1, keep_prob)
    
    # Layer 4: Fully Connected. Input = 120. Output = 84.
    w4 = tf.Variable(tf.truncated_normal(shape=(120,84), mean = mu, stddev = sigma))
    b4 = tf.Variable(tf.zeros(84))
    fc2 = tf.add(tf.matmul(drop1,w4), b4)
    fc2 = tf.nn.relu(fc2)
    
    # Dropout 2
    drop2 = tf.nn.dropout(fc2, keep_prob)    
    
    # TODO: Layer 5: Fully Connected. Input = 84. Output = 43.
    w5 = tf.Variable(tf.truncated_normal(shape=(84,n_classes), mean = mu, stddev = sigma))
    b5 = tf.Variable(tf.zeros([n_classes]))
    logits = tf.add(tf.matmul(drop2,w5), b5)
    
    return logits
    

### Question 3

_What does your final architecture look like? (Type of model, layers, sizes, connectivity, etc.)  For reference on how to build a deep neural network using TensorFlow, see [Deep Neural Network in TensorFlow
](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/b516a270-8600-4f93-a0a3-20dfeabe5da6/concepts/83a3a2a2-a9bd-4b7b-95b0-eb924ab14432) from the classroom._


**Answer:**

- A convolutional layer with a 5x5 filter, 1x1 stride, VALID padding, and depth of 6. The output is 28x28x6
- A relu activation
- A pooling layer with 2x2 kernel and 2x2 stride. The output is 14x14x6
- A convolutional layer with a 5x5 filter, 1x1 stride, VALID padding, and depth of 16. The output is 10x10x6
- A relu activation 
- A pooling layer with 2x2 kernel and 2x2 stride. The output is 5x5x6
- flatten the output from previous layer into a vector 5x5x16 = 400
- A fully connected layer with width of 120
- A dropout layer with keep probablity of 0.5
- Another fully connected layer with width of 84
- A dropout layer with keep probablity of 0.5
- An output layer with 43 width (number of classes)


In [7]:
### Train your model here.
### Feel free to use as many code cells as needed.
x = tf.placeholder(tf.float32, (None, 32, 32, 3))
y = tf.placeholder(tf.int32, (None))
keep_prob = tf.placeholder(tf.float32)
one_hot_y = tf.one_hot(y, n_classes)

rate = 0.001

logits = LeNet(x, keep_prob)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, one_hot_y)
loss_operation = tf.reduce_mean(cross_entropy)
#optimizer = tf.train.AdamOptimizer(learning_rate = rate)
optimizer = tf.train.AdamOptimizer()
training_operation = optimizer.minimize(loss_operation)


#model evaluation
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.0})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples


#train the model
accuracy = []
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: KEEP_PROB})
            
        validation_accuracy = evaluate(X_validation, y_validation)
        accuracy.append(validation_accuracy)
        print("EPOCH {} ...".format(i+1), "Validation Accuracy = {:.3f}".format(validation_accuracy))
        
    test_accuracy = evaluate(X_test, y_test)
    print("Test Accuracy = {:.3f}".format(test_accuracy))
    
    saver.save(sess, 'lenet')
    print("Model saved")
    
#plot accuracy over epochs
plt.plot(range(len(accuracy)), accuracy)
plt.ylim([0.7,1])
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.show

Training...

EPOCH 1 ... Validation Accuracy = 0.448
EPOCH 2 ... Validation Accuracy = 0.496
EPOCH 3 ... Validation Accuracy = 0.517
EPOCH 4 ... Validation Accuracy = 0.532
EPOCH 5 ... Validation Accuracy = 0.539
EPOCH 6 ... Validation Accuracy = 0.565
EPOCH 7 ... Validation Accuracy = 0.571
EPOCH 8 ... Validation Accuracy = 0.569
EPOCH 9 ... Validation Accuracy = 0.586
EPOCH 10 ... Validation Accuracy = 0.581


InvalidArgumentError: Incompatible shapes: [32] vs. [32,10]
	 [[Node: Equal_1 = Equal[T=DT_INT64, _device="/job:localhost/replica:0/task:0/cpu:0"](ArgMax_2, ArgMax_3)]]

Caused by op 'Equal_1', defined at:
  File "/Users/vincechan/anaconda/envs/carnd-term1/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/vincechan/anaconda/envs/carnd-term1/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/vincechan/anaconda/envs/carnd-term1/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/vincechan/anaconda/envs/carnd-term1/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/vincechan/anaconda/envs/carnd-term1/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/Users/vincechan/anaconda/envs/carnd-term1/lib/python3

In [ ]:
# evaluate the model with test data
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))

    test_accuracy = evaluate(X_test, y_test)
    print("Test Accuracy = {:.3f}".format(test_accuracy))


### Question 4

_How did you train your model? (Type of optimizer, batch size, epochs, hyperparameters, etc.)_


**Answer:**

I used AdamOptimizer with a learning rate of 0.001. I had experimented with lowering learning rate, it requires a much larger epochs and as a result, much longer training time with only marginal improvement.

I used 100 epochs. I had experiemented with much larger epochs (>1000) combined with lower learning rate, it resulted with some marginal validation accuracy, but much longer training time. 

I used two drop out layers with 0.5 keep probability, one after each fully connected layer. The dropout layers are added to prevent overfitting.

I used a batch size of 32. I picked 32 mainly due to the limitation of my machine which only has a 1GB of GPU memory. However, I did experiment with much larger batch size (up to 4096) when I used Amazon EC2 instance. I found that it improved the training time greatly, but didn't improve validation accuracy.

### Question 5


_What approach did you take in coming up with a solution to this problem? It may have been a process of trial and error, in which case, outline the steps you took to get to the final solution and why you chose those steps. Perhaps your solution involved an already well known implementation or architecture. In this case, discuss why you think this is suitable for the current problem._

**Answer:**
The entire solution is the result of many trial and errors. 

For my first attempt, I simply adapted the LeNet Lab to this problem. I had to adjust the input from depth of 1 (grayscale in MINIST) to depth of 3 (color in traffic signs). I also had to adjust the final layer to n_classes (43) from 10. This alone got the validation accuracy to 96%. 

Next, I did many experiements by adjusting various hyper parameters on the existing model. When I used 500 epochs and I noticed the validation accuracy started to go down after 200 epochs. I suspected the model overshot optimum. 
I adjusted the learning rate from 0.001 to 0.0001. After this adjustment, with 0.0001 learning rate and 500 epochs, the validation accuracy increased to 99%.

<img src="500epochs.png"/>


Next I experiemented changing the architecture. The first idea I had was related to the padding. The first conv layer from LeNet lab was cutting off the edges. The first layer has an input of 32x32 and output of 28x28. This cutoff was fine for MINIST in the LeNet lab where the original images was smaller and had zero padded around the edges. I thought if there are useful features near the edges in the traffic sign images, they would get cut off. Based on that idea, I updated the first conv layer to use 'SAME' instead of 'VALID' padding. This will give an output of 32x32 after the first conv layer. However, that didn't produce any noticeable improvement. I also experimented with different depths in the conv layers without noticeable improvement.

Next, I experiemented with the batch size. On my local machine, my GPU only has 1GB memory, and I couldn't use anything bigger than. To get around that limiation, I started using an Amazon EC2 gp2 instance. On the Amazon instance, I used 4096 as the batch size. The training time was a lot faster, but the accuracy wasn't improving.

Next, reasonably satifitied with the 99% valiation accuracy, I gave the test data a shot. The test accuracy was ~92%, quite a big gap from the validation accuracy. This clearly indicated overfitting. Also, this could also be a result of deformed images in the test set that the trained model couldn't predict.

Equipped with what I learnt from the last experiment. Next, I added preprocessing. I normalized the data. I also generated additional training data. I also noticed there are many classes that are very under represented in the training data. I generated the additional data for those classes by rotating the images. I also added two dropout layers, one after each fully connected layer to combat overfitting. With the new pipeline, I could increase the learning rate, decrease the epochs and speed up the training, and achieved a 99.2% validation accuracy and 95% test accuracy.


---

## Step 3: Test a Model on New Images

Take several pictures of traffic signs that you find on the web or around you (at least five), and run them through your classifier on your computer to produce example results. The classifier might not recognize some local signs but it could prove interesting nonetheless.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.
import matplotlib.image as mpimg
import numpy as np
import os
import cv2

extras = [
    { "file": "img1.png", "image": None, "pred_label": None, "label": 13, "sign": "Yield" },
    { "file": "img2.png", "image": None, "pred_label": None, "label": 14, "sign": "Stop" },
    { "file": "img3.png", "image": None, "pred_label": None, "label": 17, "sign": "No Entry" },
    { "file": "img4.png", "image": None, "pred_label": None, "label": 18, "sign": "General Caution" },
    { "file": "img5.png", "image": None, "pred_label": None, "label": 28, "sign": "Children Crossing" },
    { "file": "img6.png", "image": None, "pred_label": None, "label": -1, "sign": "Railroad Crossing" },    
];

for extra in extras:
    image = mpimg.imread("extra/" + extra["file"])
    image = image[:,:,:3]
    extra["image"] = cv2.resize(image, (32,32))    

### Question 6

_Choose five candidate images of traffic signs and provide them in the report. Are there any particular qualities of the image(s) that might make classification difficult? It could be helpful to plot the images in the notebook._



**Answer:**

In [ ]:
### Run the predictions here.
### Feel free to use as many code cells as needed.
import tensorflow as tf

# run predictions
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))   
    pred = sess.run(tf.argmax(logits, 1), feed_dict = { x: [e["image"] for e in extras], keep_prob: 1.0})

# store predictions in extras 
for i in range(len(pred)):
    extras[i]["pred_label"] = pred[i]

# plot result
fig = plt.figure(figsize = (10,6))
for idx, extra in enumerate(extras):
    ax = fig.add_subplot(len(extras), 3, idx * 3 + 1)
    ax.imshow(extra["image"])
    ax.set_xticks(())
    ax.set_yticks(())
    correctness = "CORRECT" if extra["label"] == extra["pred_label"] else "WRONG"
    ax.text(40,20,"Prediction: {0}\nPredicted Label: {1}\nCorrect Label: {2}".format(correctness, extra["pred_label"], extra["label"]))
    
    ax = fig.add_subplot(len(extras), 3, idx * 3 + 2)
    ax.imshow(sample_data[extra["pred_label"]][1])
    ax.set_xticks(())
    ax.set_yticks(())
    ax.text(40,20,"Predicted\n{0}".format(sample_data[extra["pred_label"]][0]))
    
    ax = fig.add_subplot(len(extras), 3, idx * 3 + 3)
    ax.set_xticks(())
    ax.set_yticks(())    
    correct_label = extra["label"]
    if correct_label == -1:
        ax.imshow(np.zeros((32,32,3)))
        ax.text(40,20,"Actual\n{0}\n(Not in set)".format(extra["sign"]))
    else:
        ax.imshow(sample_data[extra["label"]][1])
        ax.text(40,20,"Actual\n{0}".format(sample_data[extra["label"]][0]))
plt.tight_layout()    




The are two images that model did not predict correctly. 

Children crossing. 
This sign is in the training set, but there are quite a few differences with the extra image that could throw off the model. First, The shape of the sign in the training set is triangle, with thick red border. The shape of the sign in the extra image is pentagon, and it's entirely yellow with much thinner border. Also, the shape of the two people in the signs are different. Moreover, the extra image has a sentence at the bottom that's absent in the training set.

Railroad crossing. 
This image is not in the training set, so the model could never have predicted correctly.

### Question 7

_Is your model able to perform equally well on captured pictures when compared to testing on the dataset? The simplest way to do this check the accuracy of the predictions. For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate._

_**NOTE:** You could check the accuracy manually by using `signnames.csv` (same directory). This file has a mapping from the class id (0-42) to the corresponding sign name. So, you could take the class id the model outputs, lookup the name in `signnames.csv` and see if it matches the sign from the image._


**Answer:**

On the new images, the accuracy is 66.7%. This is not nearly as good as the 95% test accuracy. This is partially due to the railroad crossing image that's not in the training set, and the differences in the children crossing image.


In [ ]:
### Visualize the softmax probabilities here.
### Feel free to use as many code cells as needed.
import tensorflow as tf

# run top_k on softmax
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))   
    topk = sess.run(tf.nn.top_k(tf.nn.softmax(logits), k=5), feed_dict = { x: [e["image"] for e in extras], keep_prob: 1.0})
# print(topk)

# store predictions to plot later
for i in range(len(extras)):
    extras[i]["top1img"] = sample_data[topk[1][i][0]][1]
    extras[i]["top1pct"] = round(topk[0][i][0] * 100, 2)
    extras[i]["top2img"] = sample_data[topk[1][i][1]][1]
    extras[i]["top2pct"] = round(topk[0][i][1] * 100, 2)
    extras[i]["top3img"] = sample_data[topk[1][i][2]][1]
    extras[i]["top3pct"] = round(topk[0][i][2] * 100, 2)
    
#for e in extras:
#    print(e["topk"], e["topkpct"])

# plot result
fig = plt.figure(figsize = (10,6))            
for idx, extra in enumerate(extras):
    ax = fig.add_subplot(len(extras), 4, idx * 4 + 1)
    ax.imshow(extra["image"])
    ax.set_xticks(())
    ax.set_yticks(())
    correctness = "CORRECT" if extra["label"] == extra["pred_label"] else "WRONG"
    ax.text(40,20,"{0}\nPredicted : {1}\nCorrect : {2}".format(correctness, extra["pred_label"], extra["label"]))
    
    ax = fig.add_subplot(len(extras), 4, idx * 4 + 2)
    ax.imshow(extra["top1img"])
    ax.set_xticks(())
    ax.set_yticks(())
    ax.text(40,20,"Percentage\n{0}".format(extra["top1pct"]))
    
    ax = fig.add_subplot(len(extras), 4, idx * 4 + 3)
    ax.imshow(extra["top2img"])
    ax.set_xticks(())
    ax.set_yticks(())
    ax.text(40,20,"Percentage\n{0}".format(extra["top2pct"]))
    
    ax = fig.add_subplot(len(extras), 4, idx * 4 + 4)
    ax.imshow(extra["top3img"])
    ax.set_xticks(())
    ax.set_yticks(())
    ax.text(40,20,"Percentage\n{0}".format(extra["top3pct"]))
plt.tight_layout()
            


### Question 8

*Use the model's softmax probabilities to visualize the **certainty** of its predictions, [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. Which predictions is the model certain of? Uncertain? If the model was incorrect in its initial prediction, does the correct prediction appear in the top k? (k should be 5 at most)*

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

**Answer:**

The model predicted correctly with 100% certainty on 3 images!

The model predicted the stop sign correctly at 99.32%. This is a bit less cerain than the other 3 correctly predicted image. This is possibly due to the background in the stop sign in the extra image. It has a much busier background compared to other images.

The model predicated the childred crossing sign incorrectly with a low certainty at 50.23%. This is a local sign that has many differences with the sign in the training set, so the low certainty makes sense. Interestingly, the next two highest probable prediction didn't include the correct sign. I ran top_k to include all predictions and found that the correct result came in at number 12 with 0.7% certainty.

The model predicated the railroad crossing sign incorrectly with a high certainly at 94.77%. This image is not in the training data so the model could never have predicatd it correctly. The model predicated it as the 30 km/h speed limit sign. There are some similarities between both signs. They both have have circular border. They both have two  characters in the middle. However, they look quite different visually. I am surprised the model predicated with such high certainty.


> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.